<a href="https://colab.research.google.com/github/popkaotarbuza/sleep-disorder-diagnosis/blob/dev/ABC-analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
github_csv_url = 'https://raw.githubusercontent.com/popkaotarbuza/sleep-disorder-diagnosis/refs/heads/dev/src/data/Sleep_health_and_lifestyle_dataset.csv'
df = pd.read_csv(github_csv_url)

In [42]:
# Получаем все профессии
all_occupations = df['Occupation'].unique()

# Приводим все Series к одному индексу (чтобы корректно суммировались счетчики людей с расстройством сна)
with_sleep_apnea = df[df['Sleep Disorder'] == 'Sleep Apnea'].groupby('Occupation').size().reindex(all_occupations, fill_value=0)
with_insomnia    = df[df['Sleep Disorder'] == 'Insomnia'].groupby('Occupation').size().reindex(all_occupations, fill_value=0)
total            = df.groupby('Occupation').size().reindex(all_occupations, fill_value=0)

# Создаём таблицу датафрейма
disorder_detail = pd.DataFrame({
    'Всего человек': total,
    'Апноэ сна': with_sleep_apnea,
    'Бессонница': with_insomnia,
    'Без расстройств': total - (with_sleep_apnea + with_insomnia)
})

# Добавляем метрики
disorder_detail['С расстройством сна'] = disorder_detail['Апноэ сна'] + disorder_detail['Бессонница']
disorder_detail['% с расстройством'] = (disorder_detail['С расстройством сна'] / disorder_detail['Всего человек'] * 100).round(1)

# Сортируем по убыванию больных
disorder_detail = disorder_detail.sort_values('С расстройством сна', ascending=False)

# ABC-анализ
total_cases = disorder_detail['С расстройством сна'].sum()
disorder_detail['Доля от всех случаев, %'] = (disorder_detail['С расстройством сна'] / total_cases * 100).round(20)
disorder_detail['Накопительная доля, %'] = disorder_detail['Доля от всех случаев, %'].cumsum()

#A - высокая группа риска
#B - средняя группа риска
#C - низкая группа риска
def abc_group(cum):
    if cum <= 80:
        return 'A'
    elif cum <= 95:
        return 'B'
    else:
        return 'C'

disorder_detail['ABC-группа'] = disorder_detail['Накопительная доля, %'].apply(abc_group)

# Сбрасываем индекс — теперь колонка с профессией называется 'Occupation'
disorder_detail = disorder_detail.reset_index()

# Переименовываем колонку в "Профессия"
disorder_detail = disorder_detail.rename(columns={'Occupation': 'Профессия'})

# Финальная таблица
final = disorder_detail[[
    'Профессия', 'Всего человек', 'Апноэ сна', 'Бессонница', 'Без расстройств',
    '% с расстройством', 'Накопительная доля, %', 'ABC-группа'
]]
print(final.to_string(index=False))

           Профессия  Всего человек  Апноэ сна  Бессонница  Без расстройств  % с расстройством  Накопительная доля, % ABC-группа
               Nurse             73         61           3                9               87.7              41.290323          A
             Teacher             40          4          27                9               77.5              61.290323          A
         Salesperson             32          1          29                2               93.8              80.645161          B
              Doctor             71          4           3               64                9.9              85.161290          B
          Accountant             37          0           7               30               18.9              89.677419          B
            Engineer             63          1           5               57                9.5              93.548387          B
              Lawyer             47          3           2               42               10.6   